### Problem 1: Optimise and suggest a salad combination based on customer demands to yield maximum profits

In [ ]:
import pandas as pd
import numpy as np
from rsome import ro
from rsome import grb_solver as grb

In [11]:
"""
Data Preparation
"""

ORIGINAL_QTY = 5
data1 = pd.read_csv('menu.csv')
data1['Total'] = ORIGINAL_QTY
header1 = data1.columns

print("Shape of dataset: {} rows, {} columns".format(data1.shape[0], data1.shape[1]))
print("Total number of ingredients is {} and should be {}".format(sum(data1['Total']), data1.shape[0] * ORIGINAL_QTY))
data1.head()

Shape of dataset: 88 rows, 26 columns
Total number of ingredients is 440 and should be 440


,Ingredient,Ingredient_type,Price,Serving_size,COGS,"per (g, pcs, unit)",COGS_per_serving,Adjusted_COGS_per_serving (if needed),Calories,Carbohydrates,...,Carbon_footprint,Vegan,Vegetarian,Gluten,Dairy,Nuts,Spicy,Sources,Unnamed: 24,Total
0,Red & White Cabbage,Standard Base,NaN,90,6.45,1100.0,0.53,NaN,26,3.4,...,0.064,1,1,0,0,0,0,https://omni.fairprice.com.sg/category/fruits-...,NaN,5
1,Romaine,Standard Base,NaN,120,3.10,200.0,1.86,0.86,20,4.0,...,0.033,1,1,0,0,0,0,https://omni.fairprice.com.sg/search?query=rom...,NaN,5
2,Kale,Premium Base,NaN,90,5.00,120.0,3.75,1.75,32,4.0,...,0.022,1,1,0,0,0,0,https://omni.fairprice.com.sg/product/vegeponi...,NaN,5
3,Baby Spinach,Premium Base,NaN,90,1.25,200.0,0.56,1.56,21,3.3,...,0.029,1,1,0,0,0,0,https://omni.fairprice.com.sg/product/kok-fah-...,NaN,5
4,Wholemeal Wrap,Wrap,NaN,100,3.55,360.0,0.99,NaN,297,54.8,...,0.180,0,1,0,0,0,0,https://www.fairprice.com.sg/product/fairprice...,NaN,5


In [12]:
"""
Generate salad based on user's input:

Details on Salad Stop:
Link: https://www.saladstop.com.sg/cyo/
1. Choose either premium base or standard base (standard base includes wrap and grain bowls)
2. Choose 2 dressing
3. Choose 7 standard toppings (even if you chose premium base)
4. Can add as many premium toppings as you want but will have additional charges
"""

def generate_salad(user_input, data_input, ingredient_qty, exclusion_list):
    
    try:
    
        '''
        Construct Optimizer.
        '''
        model = ro.Model('Salad selector model')

        '''
        Other variables to be used later:
        n refers to the total number of ingredients offered by salad stop
        '''
        n = len(data_input["ingredient"])

        '''
        Initialize Decision Variables
        x is the selection of an ingredient in the salad (binary variables)
        s is the standard base selection (binary variable)
        t is the premium base selection (binary variable)
        '''
        x = model.dvar(n, vtype='B')
        s = model.dvar((1,), vtype='B')
        t = model.dvar((1,), vtype='B')

        '''
        Create Objective Function:
        To maximize the profit to be earned by Salad Stop while meeting the constraints of the customer.
        '''
        model.max((9.9*s + 11.9*t + sum(x[i]*data_input["price"][i] for i in range(n) if data_input["ingredient_type"][i] in ['Premium Topping'])) - (sum(x[i]*data_input["cost"][i] for i in range(n))))

        '''
        Constraints 1: Optimizer will return same output given the same constraints indicated. 
        '''
        model.st(sum(x[i] for i in range(n) if ingredient_qty[i] == 0) == 0)

        '''
        Constraints 2: Depending on the base selected, ingredients should be of the same category as the type of base selected
        '''
        model.st(sum(x[i] for i in range(n) if data_input["ingredient_type"][i] in ['Standard Base', 'Wrap', 'Grain Bowl']) == s)
        model.st(sum(x[i] for i in range(n) if data_input["ingredient_type"][i] in ['Premium Base']) == t)

        '''
        Constraints 3: Ensure that either standard base is selected or premium base selected. Cannot be neither selected or both selected
        '''
        model.st(0 <= s <= 1)
        model.st(0 <= t <= 1)
        model.st(s + t == 1)

        '''
        Constraints 4: Ensure that only exactly 7 toppings unless the user wants more will be chosen
        '''
        model.st(sum(x[i] for i in range(n) if data_input["ingredient_type"][i] in ['Standard Topping']) == 7)

        '''
        Constraints 5: Ensure that only exactly 2 dressings unless the user wants more will be chosen
        '''
        model.st(sum(x[i] for i in range(n) if data_input["ingredient_type"][i] in ['Dressing (Asian)', 'Dressing (Western)']) == 2)

        '''
        Constraints 6: Ensure that the selection of ingredients meets nutrition requirements of user
        '''
        nutrition_list = [data_input["calories"], data_input["carbs"], data_input["protein"], data_input["fat"], data_input["sugar"]]
        for j in range(len(nutrition_list)):
            nutri =  nutrition_list[j]
            model.st(user_input["min_nutrition"][j] <= sum(x[i]*nutri[i] for i in range(n)))
            model.st(sum(x[i]*nutri[i] for i in range(n)) <= user_input["max_nutrition"][j])

        '''
        Constraints 7: Ensure that the dietary needs of user is met
        '''
        reqs = [data_input["vegan"], data_input["vegetarian"], data_input["gluten"], data_input["dairy"], data_input["nuts"], data_input["spicy"]]
        for k in range(len(user_input["dietary_req"])):
            req_type = reqs[k]
            if user_input["dietary_req"][k] == 0:
                model.st(sum(x[i] for i in range(n) if req_type[i] == 1) == 0 )

        '''
        Constraints 8: Ensure that the number of premium toppings meet user requirements
        '''
        model.st(sum(x[i] for i in range(n) if data_input["ingredient_type"][i] in ['Premium Topping']) <= user_input["max_num_of_premium_toppings"])

        '''
        Constraints 9: Ensure that the total cost of the salad is within the user's budget
        '''
        model.st((9.9*s + 11.9*t + sum(x[i]*price[i] for i in range(n) if data_input["ingredient_type"][i] in ['Premium Topping'])) <= user_input["budget"])

        '''
        Constraints 10: Ensure that the total cost of the salad is within the user's budget
        '''
        model.st(x >= 0)
        
        
        '''
        Constraints 11: Randomizer
        '''
        for z in range(len(exclusion_list)):
            model.st(sum(x[y] for y in range(n) if data_input["ingredient"][y] == exclusion_list[z]) == 0)
        
        '''
        Solve Model and generate results
        '''
        model.solve(grb)
        
        if int(s.get()[0]) == 1:
            base = 9.9
        else:
            base = 11.9

        return x.get(), model.get(), base
    
    except:

        return [], 0, 0

In [13]:
"""
Set user requirements
"""

user_input = {
    "min_nutrition": np.array([450, 30, 20, 0, 0]), # calories, carbs, protein, fat, sugar
    "max_nutrition": np.array([500, 500, 30, 10, 10]),
    "budget" : 17,
    "max_num_of_premium_toppings": 3,
    "dietary_req": [1,1,1,1,1,1] # vegan, vegetarian, gluten, dairy, nuts, spicy
}

In [14]:
"""
Set dataset values
"""

data = data1.values

ingredient, ingredient_type, price, cost, calories, carbs, protein, fat, sugar =\
data[:,0], data[:,1], data[:,2], data[:,6], data[:,8], data[:,9], data[:,10], data[:,11], data[:,12]

vegan, vegetarian, gluten, dairy, nuts, spicy =\
data[:,-9], data[:,-8], data[:,-7], data[:,-6], data[:,-5], data[:,-4]

data_input = {
    "ingredient": ingredient,
    "ingredient_type": ingredient_type,
    "price": price,
    "cost": cost,
    "calories": calories,
    "carbs": carbs,
    "protein": protein,
    "fat": fat,
    "sugar": sugar,
    "vegan": vegan,
    "vegetarian": vegetarian,
    "gluten": gluten,
    "dairy": dairy,
    "nuts": nuts,
    "spicy": spicy,
}

In [15]:
"""
Randomizer to "force" different combinations
"""

randomizer = []

In [18]:
total = data[:,-1]
total[-1] = 0

for j in range(5):
    
    response, cost, base = generate_salad(user_input, data_input, total, randomizer)
    
    randomizer = []
    
    if base == 0:
        print("Cannot make a salad... Impossible set of constraints")
    
    else:
        print("\n=========== RECEIPT ===========\n")
        print("Purchase: \n")
        total_cost = 0
        total_cost += base
        for i in range(len(response)):
            if response[i] > 0:
                
                randomizer.append(ingredient[i])
                
                index = np.where(data_input["ingredient"] == ingredient[i])[0][0]
                
                print("{}, {}".format(ingredient[i], data_input["ingredient_type"][index]))
                
                if data_input["ingredient_type"][index] == "Premium Topping":
                    total_cost += data_input["price"][index]
                
                if total[i] > 0:
                    total[i] -= 1

        print("\n")
        print("Customer #{}, base price: {}".format(j, base))
        print("Cost of salad: ${}".format(round(total_cost,2)))
        print("Profit from salad: ${}".format(round(cost,2)))
        print("Amount of ingredients left: {}".format(sum(total)))
        print("\n===============================\n")

Being solved by Gurobi...
Solution status: 2
Running time: 0.1176s

=========== RECEIPT ===========

Purchase: 

Tomato Wrap, Wrap
Pea Sprouts, Standard Topping
Sesame Seeds, Standard Topping
Furikake, Standard Topping
Lime Wedge, Standard Topping
Fresh Herbs, Standard Topping
Roasted Pumpkins, Standard Topping
Cucumbers, Standard Topping
Seared Tuna, Premium Topping
Thai Turmeric, Dressing (Asian)
Smoked Pimento, Dressing (Western)


Customer #0, base price: 9.9
Cost of salad: $13.9
Profit from salad: $5.41
Amount of ingredients left: 363


Being solved by Gurobi...
Solution status: 2
Running time: 0.1089s

=========== RECEIPT ===========

Purchase: 

Quinoa, Grain Bowl
Green Apple, Standard Topping
Soba Noodles, Standard Topping
Potato, Standard Topping
Black Beans, Standard Topping
French Beans, Standard Topping
Red Onions, Standard Topping
Carrot, Standard Topping
Thai Asparagus, Premium Topping
Smoked Salmon, Premium Topping
Whole Eggs, Premium Topping
Salt & Pepper, Dressing (Wes

In [20]:
# How to do it such that theres different combinations of food everytime? (randomizer?)
# Add more constraints to increase customizability/randomness of output (e.g. Carbon Footprint)

### Problem 2: Optimise order of ingredients based on historical demand

In [110]:
from sklearn.model_selection import train_test_split
import random

Demand_data = pd.read_csv("demand_menu.csv")
d = Demand_data.set_index('Date').T
demand_train, demand_test = train_test_split(d, test_size=0.3)
demand_train

Date,Red & White Cabbage,Romaine,Kale,Baby Spinach,Wholemeal Wrap,Spinach Wrap,Tortilla Wrap,Tomato Wrap,Cauliflower Rice,Quinoa,...,Cashew Mint,Classic Caesar,Honey Dijon,Lemon & Oil,Mixed Berries Vinaigrette,Olive Oil,Salt & Pepper,Smoked Pimento,Smoky Ranch,Tabbasco Sauce
5/1/22,90,66,98,24,100,38,24,99,92,12,...,99,54,94,87,99,57,38,5,86,61
2/1/22,78,58,30,43,69,66,88,15,96,64,...,45,21,60,69,36,40,1,93,26,5
4/1/22,57,7,97,16,44,61,95,31,36,85,...,14,89,81,65,15,29,14,70,14,8
7/1/22,50,9,74,78,64,45,96,4,25,95,...,16,45,34,7,35,2,29,46,29,20


In [114]:
"""
9.9: 2 Dressing + 7 Ingredient + 1 Standard Base
11.9: 2 Dressing + 7 Ingredient + 1 Premium Base

Assume that the price of each element is constant, hence:
1 Dressing: 0.99
1 Ingredient: 0.99
1 Standard Base: 0.99

Then Premium base would be the cost of standard base + the difference between premium base and standard base, hence:
1 Premium Base: 2.99
"""

Param_data = pd.DataFrame().assign(Ingredient=data1['Ingredient'], COGS=data1['COGS_per_serving'], Ingredient_Type=data1['Ingredient_type'], Additional_Price_For_Premium_Toppings=data1['Price'])
Param_data['Price'] = None
Param_data['Space'] = None

for i in range(Param_data.shape[0]):
    if Param_data["Ingredient_Type"][i] in ["Standard Base", "Wrap", "Grain Bowl", "Standard Topping", "Dressing (Western)", "Dressing (Asian)"]:
        Param_data["Price"][i] = 0.99
    elif Param_data["Ingredient_Type"][i] in ["Premium Base"]:
        Param_data["Price"][i] = 2.99
    elif Param_data["Ingredient_Type"][i] in ["Premium Topping"]:
        Param_data["Price"][i] = 0.99 + Param_data["Additional_Price_For_Premium_Toppings"][i]
        
    Param_data["Space"][i] = round(random.uniform(0, 1), 2)
        
Param_data = Param_data.set_index('Ingredient').T
Param_data

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value i

Ingredient,Red & White Cabbage,Romaine,Kale,Baby Spinach,Wholemeal Wrap,Spinach Wrap,Tortilla Wrap,Tomato Wrap,Cauliflower Rice,Quinoa,...,Classic Caesar,Honey Dijon,Lemon & Oil,Mixed Berries Vinaigrette,Olive Oil,Salt & Pepper,Smoked Pimento,Smoky Ranch,Tabbasco Sauce,Placeholder
COGS,0.53,1.86,3.75,0.56,0.99,1.4,0.79,0.67,1.47,1.01,...,0.83,0.75,1.19,0.75,0.34,0.29,1.07,0.85,0.25,0
Ingredient_Type,Standard Base,Standard Base,Premium Base,Premium Base,Wrap,Wrap,Wrap,Wrap,Grain Bowl,Grain Bowl,...,Dressing (Western),Dressing (Western),Dressing (Western),Dressing (Western),Dressing (Western),Dressing (Western),Dressing (Western),Dressing (Western),Dressing (Western),Placeholder
Additional_Price_For_Premium_Toppings,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Price,0.99,0.99,2.99,2.99,0.99,0.99,0.99,0.99,0.99,0.99,...,0.99,0.99,0.99,0.99,0.99,0.99,0.99,0.99,0.99,None
Space,0.9,0.31,0.64,0.52,0.52,0.68,0.71,0.61,0.38,0.25,...,0.04,0.34,0.8,0.01,0.7,0.62,0.09,0.68,0.45,0.35


In [117]:
Demand = np.array( demand_train.values[:,1:demand_train.shape[1]].astype(int) )
price = np.array( Param_data.values[3,1:demand_train.shape[1]].astype(np.float64) )
cost = np.array( Param_data.values[0,1:demand_train.shape[1]].astype(np.float64) )
space = np.array( Param_data.values[4,1:demand_train.shape[1]].astype(np.float64) )
total_space = 300

In [118]:
def Multi_Newsvendor_Emp(p,c,D,s,C):
    K,N = D.shape
    mnv = ro.Model('Multi_Newsvendor')
    x = mnv.dvar(N)
    t = mnv.dvar( (K,N) )
    mnv.max( 1/K*( (t @ p).sum() ) - c @ x )
    mnv.st( t[:,i] <= x[i] for i in range(N) )
    mnv.st( t <= D )
    mnv.st( s @ x <= C )
    mnv.st( x >= 0)
    mnv.solve(grb, display = True)
    return x.get(), mnv.get()

In [119]:
order_emp, profit_emp = Multi_Newsvendor_Emp(price,cost,Demand,space,total_space)
print('The optimal order quantity is:', order_emp )
print('-------------------------------------')
print('And the corresponding expected profit is:', profit_emp )

Being solved by Gurobi...
Solution status: 2
Running time: 0.0037s
The optimal order quantity is: [ 0.          0.         24.          0.          0.          0.
  0.          0.          0.         29.          0.         67.
  0.         62.          0.         89.          0.          0.
  0.          0.          0.          0.         56.          0.
  0.         59.          0.          0.         63.          0.
  0.          0.         12.          0.          0.          0.
  0.          0.          0.         61.          0.          0.
  0.         45.         57.          0.          7.         21.
 69.         45.          7.          0.         10.          0.
 78.          0.          0.          0.         40.          0.
 32.51162791  0.         25.         39.         18.          0.
  0.          0.          0.          0.          0.          0.
  0.          3.          0.          0.          0.         21.
  0.          0.         15.          0.          0.     